In [81]:
import os
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
import glob
import numpy as np
import functools
import pickle
from tqdm import tqdm_notebook as tqdm

In [18]:
def positionIndex(word_list, docid, position_index_dict, visitedfiles, completedfiles):
        
        for word in word_list:
            
            indices = [i for i, x in enumerate(word_list) if x == word]
            gaps = [j - i for i, j in zip(indices[:-1], indices[1:])] 
            gaps = [indices[0]]+ gaps
            
            print(word)
            print(position_index_dict)
            
            if (position_index_dict.get(word)) != None and (docid not in completedfiles) :
                newdata = (docid,len(gaps),gaps)
                if not newdata in position_index_dict.get(word):
                    position_index_dict.get(word).append((docid,len(gaps),gaps))

            else:
                if not position_index_dict.get(word):
                    position_index_dict[word] = [(docid,len(gaps),gaps)]
                    if docid not in visitedfiles:
                        visitedfiles.append(docid)
        
        return visitedfiles, completedfiles, position_index_dict

In [39]:
def positionIndex(word_list, docid, position_index_dict, visitedfiles, completedfiles):
    
    #print('\n\n')
    #print(docid)
    
    #{Term:{Document:[occurances]}}
    
    
    for word in word_list:
        
        indices = [i for i, x in enumerate(word_list) if x == word]
        
        if word in position_index_dict:
            if docid in position_index_dict[word]:
                position_index_dict[word][docid] = indices
            else:
                position_index_dict[word].update({docid: indices})
            
        else:
            position_index_dict.update({word:{docid:indices}})

    
    return position_index_dict

In [130]:
def createPositionIndex():
    
    position_index_dict = {}
    visitedfiles = []
    completedfiles = []
    
    for filename in tqdm(glob.glob("corpus/*.txt")):
        fo = open(filename, "r")
        data = fo.read()
        tokens = nltk.word_tokenize(data)
        unigramlist = nltk.word_tokenize(data)
        
        position_index_dict = positionIndex(unigramlist,
                                            (os.path.basename(filename).split(".")[0]),
                                             position_index_dict, visitedfiles, completedfiles)
        
        completedfiles.append((os.path.basename(filename).split(".")[0]))
        
        #print(filename," done")
        
    return position_index_dict


def check_sequence(rel_docs, query_words, position_index_dict):
        
    check_list = []
    
    prev_word = query_words[0]
    
    for doc_id in rel_docs:
        
        doc_flag = True
        
        for word in query_words:
             
            if not min(position_index_dict[prev_word][doc_id]) <= max(position_index_dict[word][doc_id]):
                
                doc_flag = False

            prev_word = word

        if doc_flag: check_list.append(True)
        else: check_list.append(False)
        

    return [doc for i, doc in enumerate(rel_docs) if check_list[i]]



def exact_match(query, position_index_dict, check_seq_flag = False):
    
    query_words = query.split(' ')
    return_doc_list = []
    
    for word in query_words:
        
        if word in position_index_dict:
            
            return_doc_list.append(set(position_index_dict[word].keys())) 
            
    rel_docs = functools.reduce(lambda s1, s2: s1 & s2, return_doc_list)
    
    if check_seq_flag: 
        
        rel_docs = check_sequence(rel_docs, query_words, position_index_dict)
        
    
    return rel_docs



def best_match(query, position_index_dict, check_seq_flag = False):
    
    query_words = query.split(' ')
    return_doc_list  = []
            
    for word in query_words:
    
        if word in position_index_dict:
            
            return_doc_list.extend(position_index_dict[word].keys()) 
            
    return set(return_doc_list)


def check_proximity(rel_docs, query_words, position_index_dict, N = 3):
    
    doc_flag = True
    

    for doc in rel_docs:
        
        for word in query_words:
            
            if not min(position_index_dict[prev_word][doc_id] - position_index_dict[word][doc_id]):

                position_index_dict[doc][word]
            
        prev_word = word
    
    
    if doc_flag: check_list.append(True)
    else: check_list.append(False)
    
    return 


def proximity_match(query, position_index_dict, N = 3, check_seq_flag = False):
    
    #Ordered best match within proximity N search: same as above but order
    #matters and any two query terms should separated by no more than N other
    #tokens.
    
    query_words = query.split(' ')
    return_doc_list  = []
    
    for word in query_words:
    
        if word in position_index_dict:
            return_doc_list.extend(position_index_dict[word].keys()) 
    
    return_doc_list = list(set(return_doc_list))
    
    check_proximity(return_doc_list, query_words, position_index_dict, N = 3)
    
    return

In [115]:
position_index_dict = createPositionIndex()

A Jupyter Widget

In [129]:
exact_match('hello world yo', position_index_dict, True)

hello
True
world
True
yo
True


['CACM-3206']

In [131]:
best_match('hello world yo', position_index_dict, True)

{'CACM-1698',
 'CACM-1769',
 'CACM-1856',
 'CACM-2311',
 'CACM-2572',
 'CACM-2957',
 'CACM-2999',
 'CACM-3077',
 'CACM-3146',
 'CACM-3205',
 'CACM-3206'}